In [1]:
import pandas as pd
import os
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
import re
from tqdm.auto import tqdm

import matplotlib.pyplot as plt

In [14]:
def new_sample(sample):
    try:
        if sample[-2:] == '. ':
            new_sample = sample

        elif sample[-1] == '.':
            new_sample = sample+' '

        elif sample[-1] != '.':
            new_sample = sample+'. '

        else:
            print('please check the data!')
    except:
        new_sample = sample
        
    return new_sample

In [15]:
def split_sent(text:str)->list:
    # split 시 무시할 패턴 정의    
    # 숫자. 로 끝나는 경우(숫자 .제외)보다 숫자. 로 시작하는 경우가 많다고 가정함
    #check = '(eqs?\.)|(figs?\.)|(sec\.)|(et\.\set\.)|(et\.?\sal\.?)|(e\.g\.)|(i\.e\.)|(no\.)|([0-9]\.)|(vs\.)|(ref\.)|(cf\.)'
    check = """ (eqs?\.)|(figs?\.)|(sec\.)|(et\.\set\.)|(et\.?\sal\.?)|(e\.g\.)|(i\.e\.)|(no\.)\
      |^([0-9]\.)$|(.?[0-9]+\.)$|(vs\.)|(ref\.)|(cf\.)\ # 논문 관련 약자들
      |(\s?\[\s?\w+\s?\w*\]) | (\<\w+\>) | (【\w+=?\w*】\s?) | ([\(\)])\ #괄호문자
      |(www\.)|(\.?\s?[a-z]+\.)|(\s?[a-z]+\.)\ # 링크주소1 www. segye. co
      |(\s+[가나다라마바사아자차카타파하]\.)
      |(\s+[ⅠⅠⅡⅢⅣⅤⅩ]+\.)
      """ 
    pp = re.compile(check, re.VERBOSE|re.IGNORECASE)
    
    # 문장부호에 특수문자가 붙거나 유니코드일 경우 처리
    text = cleansing(text)
    
    # 예외처리: 문장 끝에 .(공백없음)으로 끝나거나 .이 없는 경우
    sample = new_sample(text)
    splited_list = []
    prev_end = 0
    
    # .으로 끝나는 모든 경우를 find
    for sent in re.finditer('(\.\s?)|(\?\s)', sample):
        start = sent.span()[0]
        end = sent.span()[1]
        
        if start < 8:
            token = sample[start-start:end-1]
        else:
            token = sample[start-8:end-1]

        # '. ' 앞뒤에 check에 해당하는 패턴이 없는 경우 문장으로 나누기
        if pp.search(token) == None:
            # [0-9]+㎡, 날짜정보 예외처리 0000.00.00.
            # print(token)
            except_pattern_date_1 = re.search('([0-9]{4}\.\s?[0-9]+\.\s?[0-9]+\.?)', sample[start-8:end+7])
            # print(sample[start-4:end+7])
            except_pattern_date_21 = re.search('([0-9]{4}\.\s?[0-9]+\.\s?[0-9]+\.?)', sample[start-8:end+3])
            except_pattern_date_22 = re.search('.?([0-9]{4}\.\s?[0-9]+\.?)', sample[start-8:end])
            # print(sample[start-7:end])
            # print(sample[start-7:end+3])
            except_pattern_date_3 = re.search('.?([0-9]{4}\.\s?[0-9]+\.\s?[0-9]+\.?)', sample[start-12:end])
            print(sample[start-12:end])
            except_pattern_date_check = (except_pattern_date_1 == None)&(except_pattern_date_21 ==None)\
            &(except_pattern_date_22 ==None)&(except_pattern_date_3==None)
            
            except_pattern_etc = re.search('[0-9]+\.\s?[0-9]+㎡', sample[start-8:end+4])
            
            if (except_pattern_date_check) & (except_pattern_etc == None) :
                            
                # . 2) 처럼 뒤에 주석이 있는 경우 문장으로 나누기                
                # 주석이 있다면
                if re.search('\.\s?([0-9]+\))', sample[start:end+4]):
                    juseok_info = list(re.finditer('\.?([0-9]+\)\s*)', sample[start:end+8].strip()))
                    juseok = [ji.group() for ji in juseok_info] 
                    
                    # 주석이 2개 이상이라면(2개까지만 고려)
                    if len(juseok) > 1:
                        # print('juseok: ', juseok)
                        tmp = sample[prev_end:end+3].strip()+juseok[-1].strip()
                        # print(tmp)
                        prev_end = end+juseok_info[-1].span()[1]
                        # print(prev_end)
                    else:
                        tmp = sample[prev_end:end+3].strip()
                        prev_end = end+3
                    try:
                        if re.search('\.', tmp[-1]):
                            tmp = tmp[:-1]
                    except:
                        pass

                    
                    splited_list.append(tmp)

                # original
                else:
                    splited_list.append(sample[prev_end:end-1].strip())
                    prev_end = end
            
        elif pp.search(token) != None:
            # 마지막일 경우
            if len(sample) == end:
                splited_list.append(sample[prev_end:end-1].strip())

    return splited_list


# '.●'와 같은 특이 케이스 처리를 위해 추가(10.5)
# '６８.'특수문자 처리 안 함 
def cleansing(text):
    text = text
    cleansed_text = ''
    
    try:
    # 유니코드 문자 제거
        pattern1 = """(\\uf076)|(\xa0)"""
        p1 = re.compile(pattern1, re.VERBOSE)
        cleansed_text = p1.sub("", text)
        cleansed_text = cleansed_text.strip() 
    except:
        pass
    
    try:
        # 특수기호 앞 공백 추가
        pattern2 = """[■▦◆▲●▶▽△◇©ⓒ①②③④⑤⑥⑦®㈜※【】]  #특수문자"""
        p2 = re.compile(pattern2, re.VERBOSE)
        
        match_list = [ m.group() for m in p2.finditer(cleansed_text)]
        match_list= list(set(match_list))

        for match in match_list:
            h = Substitutable(cleansed_text)
            cleansed_text = h.sub(match, ' '+match)
    except:
        pass
    
    try:
        # 마침표 특수문자 case 처리
        pattern2 = "．"
        p3 = re.compile(pattern2)
        cleansed_text = p3.sub(".", cleansed_text)
    except:
        pass

    return cleansed_text


# ref. https://stackoverflow.com/questions/15175142/how-can-i-do-multiple-substitutions-using-regex
class Substitutable(str):
    def __new__(cls, *args, **kwargs):
        newobj = str.__new__(cls, *args, **kwargs)
        newobj.sub = lambda fro,to: Substitutable(re.sub(fro, to, newobj))
        return newobj



def new_sample(sample):
    try:
        if sample[-2:] == '. ':
            new_sample = sample

        elif sample[-1] == '.':
            new_sample = sample+' '

        elif sample[-1] != '.':
            new_sample = sample+'. '

        else:
            print('please check the data!')
    except:
        new_sample = sample
        
    return new_sample


In [16]:
def find_chapter(origi_,reg):
    spli_ = []
    for i in reg.finditer(origi_):
        spli_.append([i.group(),i.span()])
    return spli_

In [17]:
def split_chapter(origi_):
    reg = re.compile("\[\[CHAPTER\]\] +(\w+) +\[\[BODY\]\] +(\w+)-?(\w+)?")
    spli_ = find_chapter(origi_,reg)
    if spli_ != []:
        pass
    elif spli_ == []:
        reg = re.compile("CHAPTER")
        spli_ = find_chapter(origi_,reg)
    if spli_ != []:
        data_all = []
        for i in range(len(spli_)):
            chap = spli_[i][0]
            try:
                star_ = spli_[i][1][1]
                end_ = spli_[i+1][1][0]
                data_ = origi_[star_:end_].split("  ")
            except:
                star_ = spli_[i][1][1]
                data_ = origi_[star_:].split("  ")
            data_all.append([chap,data_])
    else:
        data_all = []
    return data_all

In [18]:
def to_nextsentence(df_):
    stack_all = []
    for i in range(len(df_)):
        paras = df_["paragraph"][i]
        for i2 in range(len(paras)):
            try:
                sen1 = paras[i2]
                sen2 = paras[i2+1]
                stack_all.append([sen1,sen2])
            except:
                stack_all.append(["",""])
    return pd.DataFrame(stack_all).rename(columns={0:"sen1",1:"sen2"})

In [28]:
def loading_data(path_1,reg):
    os.chdir(path_1)
    tq = tqdm(os.listdir())
    fn_list = []
    for i in tq:
        if i.endswith(".txt"):
            fn_ = reg.findall(i.replace(".","_"))[0]
            fn_list.append([fn_,i])
            print(fn_)

            f = open(path_1+"\\"+i, 'rt',encoding="iso-8859-1")
            globals ()["{}".format(fn_)] = ""
            line = f.read()
            try:
                line = bytes(line,"iso-8859-1").decode("utf-8")
            except:
                try:
                    line = bytes(line,"iso-8859-1").decode("utf-16")
                except:
                    try:
                        line = bytes(line,"iso-8859-1").decode("euc-kr")
                    except:
                        line = bytes(line,"iso-8859-1").decode("cp949")
            globals ()["{}".format(fn_)] = globals ()["{}".format(fn_)] + line
             #print(line)
            f.close()
    return fn_list

In [29]:
def first_prerpo(fn_list):
    error = []
    none_error = []
    tq = tqdm(fn_list)
    for fn_,fn_2 in tq:
        test_ = globals ()["{}".format(fn_)]
        fn_s = test_.splitlines()
        origi_ = [i.strip() for i in fn_s ]
        # rule 1 = 2칸이상을 2칸으로
        origi_ = re.sub(' {2,}','  ', " ".join(origi_))

        # chapter 별로 나누기
        data_all = split_chapter(origi_)

        if data_all != []: 
            none_error.append([fn_,fn_2])
            # dataframne 만들기
            df_data = []
            for data_ in data_all:
                chap_ = data_[0]
                paras_ = data_[1]
                sents_ = []
                for para_ in paras_:
                    sen_ = split_sent(para_)
                    ## [[ABSTRACT]] 같은거 지우기 + 길이가 0인것 지우기 + ")." 지우기
                    reg_2 = re.compile(r"\[\[\w+]\]|\)\.|\.|\):\.|〃.")
                    remove_ = [a2 for a2 in sen_ if reg_2.match(a2)]
                    if len(remove_) == 0 and len(sen_)!= 0:
                        sents_.extend(sen_)
                    else:
                        pass
                df_data.append([chap_,sents_])
            df_ = pd.DataFrame(df_data).rename(columns={0:"Chapter",1:"paragraph",2:"sentences"})

            # Next sentence 형태로 만들기
            df_next = to_nextsentence(df_)
            df_next.to_excel(save_path + "\\" + fn_ + ".xlsx")
        else:
            error.append([fn_,fn_2])
    return error


In [21]:
def nara_processing(path):

    f = open(path, 'rt',encoding="iso-8859-1")
    line = f.readlines()
    try:
        texts = [bytes(li,"iso-8859-1").decode("utf-8") for li in line]
    except:
        try:
            texts = [bytes(li,"iso-8859-1").decode("utf-16") for li in line]
        except:
            try:
                texts = [bytes(li,"iso-8859-1").decode("euc-kr") for li in line]
            except:
                try:
                    texts = [bytes(li,"iso-8859-1").decode("cp949") for li in line]
                except:
                    #texts = [bytes(li,"iso-8859-1").decode("utf-8-sig") for li in line]
                    print("error occurs")
                    return ""
                    
    # \n 제거
    texts = [re.sub('\\n', '', t) for t in texts]

    # paragraph split
    texts = split_para_nara(texts)

    # .. ). 제거
    texts = [re.sub('\.{2,}', '. ', t) for t in texts]
    texts = [re.sub('\)\.\s?', ') ', t) for t in texts]
    
    return texts



def split_para_nara(sents):
    new_page = []

    save_text = ''
    prev = 0


    for i in range(len(sents)):
        text = sents[i] # .strip()
        # A. B. 추가, $
        if re.match('(^[0-9]+\)[\s+]?)?(\s?[A-Z])', text):
            # 이전 문장 저장
            # 이전 문장에 주석이 존재할 경우 주석 포함
            f_info = re.search('(^[0-9]+\)[\s+]?)', text)

            if f_info: 
                f_info_group = re.sub('[\.\s]', '', f_info.group())
                new_page.append(save_text+'.'.join(sents[prev:i])+'.'+f_info_group)
                save_text = ''
                save_text = text[f_info.span()[1]:]+'.'

            # 주석 없는 경우
            else:
                new_page.append(save_text+'.'.join(sents[prev:i])+'.')
                save_text = ''
                save_text = text+'.'
            prev = i+1


        # 숫자. 텍스트 형태(+앞에 주석이 있을 경우 예외 추가), $
        if re.match('(^[0-9]+\)[\s+]?)?(\s?[0-9]{1})', text):     
            # 이전 문장 저장
            # 이전 문장에 주석이 존재할 경우 주석 포함
            f_info = re.search('(^[0-9]+\)[\s+]?)', text)

            if f_info: 
                f_info_group = re.sub('[\.\s]', '', f_info.group())
                new_page.append(save_text+'.'.join(sents[prev:i])+'.'+f_info_group)
                save_text = ''
                save_text = text[f_info.span()[1]:]+'.'

            # 주석 없는 경우
            else:
                new_page.append(save_text+'.'.join(sents[prev:i])+'.')
                save_text = ''
                save_text = text+'.'
            prev = i+1


        # 로마자. 텍스트 형태 $
        roma =  '(^[0-9]+\)[\s+]?)?[ⅠⅠⅡⅢⅣⅤⅩ]+'
        if re.match(roma, text):  
            # 이전 문장 저장
            # 이전 문장에 주석이 존재할 경우 주석 포함
            f_info = re.search('(^[0-9]+\)[\s+]?)', text)

            if f_info: 
                f_info_group = re.sub('[\.\s]', '', f_info.group())
                new_page.append(save_text+'.'.join(sents[prev:i])+'.'+f_info_group)
                save_text = ''
                save_text = text[f_info.span()[1]:]+'.'

            # 주석 없는 경우
            else:
                new_page.append(save_text+'.'.join(sents[prev:i])+'.')
                save_text = ''
                save_text = text+'.'

            prev = i+1


        # (1) 텍스트 형태(+앞에 주석이 있을 경우 예외 추가) $
        # (가) 텍스트 형태 추가
        if re.match('(^[0-9]+\)[\s+]?)?\s?\([0-9]\)|(^[0-9]+\)[\s+]?)?\s?\([가나다라마바사아자차카타파하]\)', text):
            # 이전 문장 저장
            # 이전 문장에 주석이 존재할 경우 주석 포함
            f_info = re.search('(^[0-9]+\)[\s+]?)', text)

            if f_info: 
                f_info_group = re.sub('[\.\s]', '', f_info.group())
                new_page.append(save_text+'.'.join(sents[prev:i])+'.'+f_info_group)
                save_text = ''

            # 주석 없는 경우
            else:
                new_page.append(save_text+'.'.join(sents[prev:i])+'.')
                save_text = ''

            prev = i


    new_page.append(save_text+' '.join(sents[prev:])) # split 사용 시: .으로 조인했었음 
    new_page = [sent for sent in new_page if sent != '.'] 


    if new_page[0][0] == '.':
        new_page = new_page[0][1:]

    return new_page

In [22]:
def second_prepro(error,path_1,save_path):
    tq = tqdm(error)
    error_2 = []
    for i,i_4 in tq:
        print(i)
        path_test = path_1 + "\\" + i_4
        nara_ = nara_processing(path_test)
        if nara_ != "":
            df_data = []
            for i2 in range(len(nara_)):
                chap_ = ""

                paras_ = nara_[i2]
                sen_ = split_sent(paras_)
                ## [[ABSTRACT]] 같은거 지우기 + 길이가 0인것 지우기 + ")." 지우기
                reg_2 = re.compile(r"\[\[\w+]\]|\)\.|\.|\):\.|〃.")
                remove_ = [a2 for a2 in sen_ if reg_2.match(a2)]
                if len(remove_) == 0 and len(sen_)!= 0:
                    df_data.append([chap_,sen_])
                else:
                    pass
            df_ = pd.DataFrame(df_data).rename(columns={0:"Chapter",1:"paragraph",2:"sentences"})
            # Next sentence 형태로 만들기
            df_next = to_nextsentence(df_)
            df_next.to_excel(save_path + "\\" + i + ".xlsx")
        else:
            error_2.append(i)
    return error_2

# Hanyang 

In [26]:
path_1 = r"C:\Users\Samsung\jupyter\Project_Medical_Legal\data\20230104\hanyang\00 변환"
reg = re.compile(r"result_([a-zA-Z0-9_]+)_")
save_path = r"C:\Users\Samsung\jupyter\Project_Medical_Legal\data\20230104\result\hanyang"

In [30]:
fn_list = loading_data(path_1,reg)

  0%|          | 0/27 [00:00<?, ?it/s]

1092
LB0026
LP0037
LP0077_Global_Labour_and_Employment
LP0095
LP0752
LP0753
LP0754
LP0756
LP0757
LP0779
LP0790
LP0794
LP0796
LP0797
LP0801
LP0804
LP0805
LP0809
LP0811
LP0869
LP0876
LP0886
LP1025
LP1111
LP1113
LP328


In [31]:
error = first_prerpo(fn_list)

  0%|          | 0/27 [00:00<?, ?it/s]

In [32]:
error

[]

# Nara

In [33]:
path_1 = r"C:\Users\Samsung\jupyter\Project_Medical_Legal\data\20230104\nara\00 변환"
reg = re.compile(r"(\w+)_")
save_path = r"C:\Users\Samsung\jupyter\Project_Medical_Legal\data\20230104\result\nara"

In [34]:
fn_list = loading_data(path_1,reg)

  0%|          | 0/281 [00:00<?, ?it/s]

LB0033_교회분쟁관계법_nara
LB0035_기술이전계약론_nara
LB0052
LB0055_신미국특허법_nara
LB0057
LB0062
LB0064
LB0074_국제사법_nara
LB0079_계약법_nara
LB0082
LB0085
LB0086
LB0088
LB0099
LB0114
LB0118_민사소송실무_nara
LB0121_지적소송실무_nara
LB0134_법학입문_nara
LB0171
LB0173
LB0212
LB0217
LB0233
LB0246
LB0251
LB0260
LB0272
LB0293
LB0298_헌법소송법_nara
LB0307
LB0327_배임죄판례백선_nara
LB0328_배임죄에대한몇가지오해_nara
LB0342
LB0349
LP0020
LP0026
LP0122
LP0153
LP0154
LP0161_부동산금융법_nara
LP0204
LP0212
LP0241
LP0243
LP0246
LP0253
LP0273
LP0277_법률가의글쓰기
LP0298_학논문작성법
LP0324_생활법률_nara
LP0333
LP0366
LP0389
LP0390
LP0391
LP0420
LP0421_금융법강의01
LP0433
LP0434
LP0477
LP0483_자본시장법상기업공시
LP0497
LP0498
LP0529
LP0533
LP0537
LP0542
LP0569
LP0717_지적재산권법_nara
LP0721_특허법3
LP0723
LP0751_민사판례연구31권_nara
LP0755_민사판례연구34_nara
LP0758_민사판례연구_nara
LP0759_민사판례연구38권_nara
LP0760_민사판례연구39권_nara
LP0762_민사판례연구40권_nara
LP0763_민사판례연구제41권_nara
LP0764_민사판례연구제42권_nara
LP0765
LP0784_행정판례연구11_nara
LP0785_행정판례연구12_nara
LP0786_행정판례연구13_nara
LP0787
LP0788
LP0789
LP0792
LP0793
LP0795
LP0806
LP080

In [35]:
error = first_prerpo(fn_list)

  0%|          | 0/281 [00:00<?, ?it/s]

In [36]:
error

[['LB0033_교회분쟁관계법_nara', 'LB0033_교회분쟁관계법_nara.txt'],
 ['LB0035_기술이전계약론_nara', 'LB0035_기술이전계약론_nara.txt'],
 ['LB0052', 'LB0052_상표법[제6판]_nara.txt'],
 ['LB0055_신미국특허법_nara', 'LB0055_신미국특허법_nara.txt'],
 ['LB0057', 'LB0057_영업비밀보호법[전면개정제3판]_nara.txt'],
 ['LB0062', 'LB0062_특허법[제6판]_nara.txt'],
 ['LB0064', 'LB0064_특허의이해[제5판]_nara.txt'],
 ['LB0074_국제사법_nara', 'LB0074_국제사법_nara.txt'],
 ['LB0079_계약법_nara', 'LB0079_계약법_nara.txt'],
 ['LB0082', 'LB0082_물권법[제15판]_nara.txt'],
 ['LB0085', 'LB0085_민법강의[제28판]_nara.txt'],
 ['LB0086', 'LB0086_민법총칙[제16판]_nara.txt'],
 ['LB0088', 'LB0088_민법총칙[제4판]_nara.txt'],
 ['LB0099', 'LB0099_채권법[제13판]_nara.txt'],
 ['LB0114', 'LB0114_민사소송법강의[제2판]_nara.txt'],
 ['LB0118_민사소송실무_nara', 'LB0118_민사소송실무_nara.txt'],
 ['LB0121_지적소송실무_nara', 'LB0121_지적소송실무_nara.txt'],
 ['LB0134_법학입문_nara', 'LB0134_법학입문_nara.txt'],
 ['LB0171', 'LB0171_의료법강의[제4판]_nara.txt'],
 ['LB0212', 'LB0212_어음수표법[개정판]_nara.txt'],
 ['LB0217', 'LB0217_주식회사법대계[제4판]_nara.txt'],
 ['LB0233', 'LB0233_회사법강의[제8판]_nara.txt'

In [37]:
error_2 = second_prepro(error,path_1,save_path)

  0%|          | 0/249 [00:00<?, ?it/s]

LB0033_교회분쟁관계법_nara
LB0035_기술이전계약론_nara
LB0052
error occurs
LB0055_신미국특허법_nara
LB0057
LB0062
LB0064
LB0074_국제사법_nara
LB0079_계약법_nara
LB0082
LB0085
LB0086
LB0088
LB0099
LB0114
LB0118_민사소송실무_nara
LB0121_지적소송실무_nara
LB0134_법학입문_nara
LB0171
LB0212
LB0217
LB0233
LB0246
LB0251
LB0260
LB0272
LB0293
LB0298_헌법소송법_nara
LB0327_배임죄판례백선_nara
LB0328_배임죄에대한몇가지오해_nara
LB0342
LB0349
LP0020
LP0026
LP0122
LP0153
LP0154
LP0161_부동산금융법_nara
LP0204
LP0212
LP0241
LP0243
LP0246
LP0253
LP0273
LP0277_법률가의글쓰기
LP0298_학논문작성법
LP0324_생활법률_nara
LP0333
LP0366
LP0389
LP0390
LP0391
LP0420
LP0421_금융법강의01
LP0433
LP0434
LP0477
LP0483_자본시장법상기업공시
LP0497
LP0498
LP0529
LP0533
LP0537
LP0542
LP0569
error occurs
LP0717_지적재산권법_nara
LP0721_특허법3
LP0723
LP0751_민사판례연구31권_nara
LP0755_민사판례연구34_nara
LP0758_민사판례연구_nara
LP0759_민사판례연구38권_nara
LP0760_민사판례연구39권_nara
LP0762_민사판례연구40권_nara
LP0763_민사판례연구제41권_nara
LP0764_민사판례연구제42권_nara
LP0765
LP0784_행정판례연구11_nara
LP0785_행정판례연구12_nara
LP0786_행정판례연구13_nara
LP0787
LP0788
LP0789
LP0792
LP0793
LP0795


In [38]:
error_2

['LB0052',
 'LP0569',
 'LP1168',
 'ME0009_의학적운동처방',
 'ME0135',
 'ME0299',
 'ME0309',
 'ME0316',
 'MK0064',
 'MK0089',
 'MK0101_부인과내분비학_pdf_nara',
 'MK0111_실전근전도50증례를통한이해_pdf_nara',
 'MK0211']

# Korea

In [39]:
path_1 = r"C:\Users\Samsung\jupyter\Project_Medical_Legal\data\20230104\4-3 고대언어검토완-20230104T085410Z-001\4-3 고대언어검토완"
reg = re.compile(r"result_([a-zA-Z0-9_]+)_")
save_path = r"C:\Users\Samsung\jupyter\Project_Medical_Legal\data\20230104\result\korea"

In [40]:
fn_list = loading_data(path_1,reg)

  0%|          | 0/43 [00:00<?, ?it/s]

LB0043
LB0131
LB0135
LB0285
LB0294
LB0297
LB0301
LP0052
LP0063
LP0151
LP0181
LP0257
LP0343
LP0381
LP0394
LP0550
LP0557
LP0716
LP1110
LP1174
ME0031
ME0034
ME0035
ME0039
ME0041
ME0045
ME0076
ME0094
ME0161
ME0172
ME0190
ME0197
ME0211
ME0236
ME0246
ME0253
ME0256
ME0274
MK0055
MK0060
MK0125
MK0159
MK0181


In [41]:
error = first_prerpo(fn_list)

  0%|          | 0/43 [00:00<?, ?it/s]

In [42]:
error

[]